In [7]:
import pandas as pd
from postgres_connect import *

In [9]:
df=pd.read_csv('../../Downloads/current_serviceability_vs_delivery_rate.csv')

In [29]:
df['pincode'] = df['pincode'].str.replace(',', '')

# Convert the 'pincode' column to integers
df['pincode'] = df['pincode'].astype(int)